In [ ]:
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def convertColr(fr, clrSpace):
    if clrSpace == "hsv":
        frm = cv2.cvtColor(fr, cv2.COLOR_BGR2HSV)
    elif clrSpace == "rgb":
        frm = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
    elif clrSpace == "gray":
        frm = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
    return frm


In [ ]:
with open("/home/dhvani/Subhiksha/Opencv/Ball_Direction_Detection/Color Range.json", "r") as f:
    clrRang = json.load(f)

In [ ]:
def time(thresholdtime, frametime):
    time = frametime/thresholdtime
    return time

In [ ]:
def displayText(frame, cntr, txt, speedval, x, y):
    # frame with Average speed value
    image = cv2.putText(
        frame,
        txt+str(speedval),
        (x, y-5),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.8,
        (0,0,255),
        2,
        cv2.LINE_AA,
    )

In [ ]:
cam = cv2.VideoCapture(2)

#To calculate frames per sec (framecount required)
frametime = 0

#To calculate distance -> prev, cur, dist required
prev = None
cur = None
average_distance = 0
instantaneous_distance =0

# read frame
while True:
    #start camera returns boolean for cam availabilty and frame
    bol, frame = cam.read()

    #cam unavailable
    if bol == False:
        print("Cam unavailable")
        break

    #Count of total frames (to find fps )
    frametime += 1
    
    ballframe = frame.copy()

    # on key press q break
    if cv2.waitKey(1) == ord("q"):
        break

    #save
    if cv2.waitKey(1) == ord("c"):
        cv2.imwrite("ballspeed.png", frame)
        break

    # convert frame to hsv
    hsvframe = convertColr(ballframe, "hsv")
    # iterate color ball ranges
    for i, color in enumerate(clrRang["balls"]):
        start, stop = clrRang["balls"][color] # holds the value start hsv range and end hsv range 

        # apply thresholding for converted hsv frame
        mask = cv2.inRange(hsvframe, np.array(start), np.array(stop)) # type: ignore

        # passing mask get contours
        contours, hir = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # when more than 1 contour available
        if len(contours) != 0:

            # Countour area for each contour
            for j, ci in enumerate(iterable=contours):
                area = cv2.contourArea(ci)

                # draw bounding box of contour if greater than thresholded area
                if area > 10000:
                    x, y, w, h = cv2.boundingRect(ci)

                    #current and previous position of the ball in the horizontal direction when current value available store previous and modify current
                    if cur != None:
                        prev = cur
                    cur = x
                    # print("when cur not none cur and  prev", cur, prev )
                                                            
                    # draw rectangle border for  ball
                    ballframe = cv2.rectangle(
                        ballframe, (x, y), (x + w, y + h), (0, 255, 0), 2
                    )
                    if prev != None and cur != None:
                        instantaneous_distance = abs(prev - cur)
      

            
            #when prev, current available;  
            if prev != None and cur != None:
                #diffrence between previous and current position of all frame is made into absolute to discard negative and summed to get average distance 
                average_distance += abs(prev - cur)
                #diffrence between previous and current position of only 2 frame is made into absolute to discard negative and summed to get instantaneous distance 
                # instDist = abs(prev - cur)

            #time calculation    
            average_time = time(30,frametime)
            instantaneous_time = time(30,1)

            #speed calculation
            average_Speed = average_distance/average_time
            instantaneous_Speed = instantaneous_distance/instantaneous_time

            displayText(ballframe, contours, "Average speed ", str(average_Speed)+" pixel/sec", 30, 100 )  
            displayText(ballframe, contours, "Instantaneous Speed", str(instantaneous_Speed)+" pixel/sec", 30, 50)
                

        # when no contour available 
        else:
           ballframe = frame
               
                 
    cv2.imshow("bounding", ballframe)


cam.release()
cv2.destroyAllWindows()

[ WARN:0@4096.916] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
